## INSTALAR 

In [1]:
## pip install pywin32
## pip instal pandas
## pip install numpy
## pip install python-dateutil
## pip install openpyxl

## Biblioteca

In [2]:
import win32com.client as win32
import os
import pandas as pd
import numpy as np
from pathlib import Path
import re
import sys
import urllib.request
from datetime import datetime
import dateutil.relativedelta
import time
import random
from openpyxl import Workbook, load_workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows
win32c = win32.constants

### Baixar arquivo da http://www.anp.gov.br

In [3]:
dir = './down_base'
try:
    os.makedirs(dir)
    print('diretorio criado')
except:
    print('diretorio já existe')

diretorio criado


In [4]:
import urllib
urllib.request.urlretrieve("http://www.anp.gov.br/arquivos/dados-estatisticos/vendas-combustiveis/vendas-combustiveis-m3.xls", "./down_base/vendas-combustiveis-m3.xls")

('./down_base/vendas-combustiveis-m3.xls',
 <http.client.HTTPMessage at 0x236dec0bb00>)

### Abre e formata Excel

In [5]:
f_path = Path.cwd()
f_name = "./down_base/vendas-combustiveis-m3.xls"
filename = f_path / f_name
# create excel object
excel = win32.gencache.EnsureDispatch('Excel.Application')
# excel can be visible or not
excel.Visible = False  # False
wb = excel.Workbooks.Open(filename) 
excel.DisplayAlerts = False
wb.DoNotPromptForConvert = True
wb.CheckCompatibility = False
derivado = wb.Sheets("Plan1").PivotTables("Tabela dinâmica1")
disel = wb.Sheets("Plan1").PivotTables("Tabela dinâmica3")



In [6]:
derivado.ColumnGrand = True
derivado.RowGrand = True

disel.ColumnGrand = True
disel.RowGrand = True

In [7]:
wb.Sheets("Plan1").Range("X66").ShowDetail = True
wb.Sheets("Plan1").Range("k146").ShowDetail = True

In [8]:
DADOS_AT = [wb.Sheets("Plan1").Range("B42").Value]

In [9]:
### FECHA TUDO
wb.Close(True)
excel.Quit()

### Configura data da atualização

In [10]:
DADOS_AT = pd.DataFrame(DADOS_AT, columns =['at_data'])

In [11]:
DADOS_AT[['val1', 'val2', 'val3', 'val4', 'val5', 'val6', 'val7', 'val8']] = DADOS_AT['at_data'].str.split(' ', expand=True)
DADOS_AT['val8'] = DADOS_AT['val8'].str.replace(".","")

In [12]:
def mes(row):  
    mes = row['val6']

    if mes == "janeiro":
        return("1")
    elif mes == "fevereiro":
        return("2")
    elif mes == "março":
        return("3")
    elif mes == "abril":
        return("4")
    elif mes == "maio":
        return("5")
    elif mes == "junho":
        return("6")
    elif mes == "julho":
        return("7")
    elif mes == "agosto":
        return("8")
    elif mes == "setembro":
        return("9")
    elif mes == "outubro":
        return("10")
    elif mes == "novembro":
        return("11")
    elif mes == "dezembro":
        return("12")
    

In [13]:
DADOS_AT['val6'] = DADOS_AT.apply(mes, axis=1)

In [14]:
DADOS_AT['data_f'] = DADOS_AT['val4'] + "/" + DADOS_AT['val6'] + "/" + DADOS_AT['val8'] + " 00:00:00"

In [15]:
DADOS_AT['data_f'] = pd.to_datetime(DADOS_AT['data_f'])

### Baixando abas criadas no excel (automático)

In [16]:
try:
    DF_DERIVADO =pd.read_excel(open('./down_base/vendas-combustiveis-m3.xls', 'rb'),
                  sheet_name='Planilha1');  
    DF_DERIVADO = DF_DERIVADO.drop(columns=['TOTAL','REGIÃO']);
except:
    DF_DERIVADO =pd.read_excel(open('./down_base/vendas-combustiveis-m3.xls', 'rb'),
                  sheet_name='Plan2');  
    DF_DERIVADO = DF_DERIVADO.drop(columns=['TOTAL','REGIÃO']);

In [17]:
try:
    DF_DISEL =pd.read_excel(open('./down_base/vendas-combustiveis-m3.xls', 'rb'),
                  sheet_name='Planilha2');  
    DF_DISEL = DF_DISEL.drop(columns=['TOTAL','REGIÃO']);
except:
    DF_DISEL =pd.read_excel(open('./down_base/vendas-combustiveis-m3.xls', 'rb'),
                  sheet_name='Plan3');  
    DF_DISEL = DF_DISEL.drop(columns=['TOTAL','REGIÃO']);

### Modelando dados do DF_DISEL_FINAL

In [18]:
DF_DISEL_FINAL = DF_DISEL.melt(id_vars=["COMBUSTÍVEL", "ANO", "ESTADO", "UNIDADE" ], 
        var_name="month", 
        value_name="volume")

In [19]:
def mes2(row):  
    mes = row['month']

    if mes == "Jan":
        return("01")
    elif mes == "Fev":
        return("02")
    elif mes == "Mar":
        return("03")
    elif mes == "Abr":
        return("04")
    elif mes == "Mai":
        return("05")
    elif mes == "Jun":
        return("06")
    elif mes == "Jul":
        return("07")
    elif mes == "Ago":
        return("08")
    elif mes == "Set":
        return("09")
    elif mes == "Out":
        return("10")
    elif mes == "Nov":
        return("11")
    elif mes == "Dez":
        return("12")

In [20]:
DF_DISEL_FINAL['month'] = DF_DISEL_FINAL.apply(mes2, axis=1)

In [21]:
DF_DISEL_FINAL['year_month'] = DF_DISEL_FINAL['ANO'].astype(str) +"/"+ DF_DISEL_FINAL['month']
DF_DISEL_FINAL = DF_DISEL_FINAL.drop(columns=['ANO','month']);

In [22]:
day_t = DADOS_AT['data_f'].iloc[0]

In [23]:
DF_DISEL_FINAL['created_at'] = day_t

In [24]:
DF_DISEL_FINAL = DF_DISEL_FINAL.rename(columns={"ESTADO": "uf", "UNIDADE": "unit", "UNIDADE": "unit", "COMBUSTÍVEL": "product"})


In [25]:
DF_DISEL_FINAL = DF_DISEL_FINAL[[

'year_month',
'uf',
'product',
'unit',
'volume',
'created_at'

]]

In [26]:
DF_DISEL_FINAL = DF_DISEL_FINAL.sort_values(by=['year_month','uf','product'], ascending=True)

In [27]:
DF_DISEL_FINAL = DF_DISEL_FINAL.reset_index(drop=True)

### Modelando dados do DF_DERIVADO_FINAL

In [28]:
DF_DERIVADO_FINAL = DF_DERIVADO.melt(id_vars=["COMBUSTÍVEL", "ANO", "ESTADO", "UNIDADE" ], 
        var_name="month", 
        value_name="volume")

In [29]:
DF_DERIVADO_FINAL['month'] = DF_DERIVADO_FINAL.apply(mes2, axis=1)

In [30]:
DF_DERIVADO_FINAL['year_month'] = DF_DERIVADO_FINAL['ANO'].astype(str) +"/"+ DF_DERIVADO_FINAL['month']
DF_DERIVADO_FINAL = DF_DERIVADO_FINAL.drop(columns=['ANO','month']);

In [31]:
DF_DERIVADO_FINAL['created_at'] = day_t

In [32]:
DF_DERIVADO_FINAL = DF_DERIVADO_FINAL.rename(columns={"ESTADO": "uf", "UNIDADE": "unit", "UNIDADE": "unit", "COMBUSTÍVEL": "product"})


In [33]:
DF_DERIVADO_FINAL = DF_DERIVADO_FINAL[[

'year_month',
'uf',
'product',
'unit',
'volume',
'created_at'

]]

In [34]:
DF_DERIVADO_FINAL = DF_DERIVADO_FINAL.sort_values(by=['year_month','uf','product'], ascending=True)

In [35]:
DF_DERIVADO_FINAL = DF_DERIVADO_FINAL.reset_index(drop=True)

### Confirmando resultado agrupado

In [36]:
pd.pivot_table(DF_DISEL_FINAL,
   index=['year_month'],
   values=['volume'],
   columns=[ 'product'],
   fill_value='',
   aggfunc=np.sum,
   margins=True,
   margins_name='Total')

volume                            \
product    ÓLEO DIESEL (OUTROS ) (m3) ÓLEO DIESEL MARÍTIMO (m3)   
year_month                                                        
2013/01                      4982.090                 63086.943   
2013/02                      4609.503                 63403.331   
2013/03                      4648.732                 67900.420   
2013/04                      2832.009                 64289.502   
2013/05                      1917.120                 68870.266   
2013/06                      1680.731                 63820.512   
2013/07                      1894.716                 70155.029   
2013/08                      2022.636                 65813.389   
2013/09                      2234.734                 64628.377   
2013/10                      2015.189                 70637.637   
2013/11                      2840.896                 67972.179   
2013/12                      4645.398                 65165.412   
2014/01                      4026.472                 66567.985   
2014/02                      3337.124                 63321.197   
2014/03                      2494.016                 63229.395   
2014/04                      2895.763                 61847.153   
2014/05                      2129.934                 67515.268   
2014/06                      2191.190                 60723.820   
2014/07                      2079.592                 67283.736   
2014/08                      2807.996                 67302.632   
2014/09                      2279.367                 70630.350   
2014/10                      2780.420                 77276.891   
2014/11                      2667.853                 64716.217   
2014/12                      3886.551                 67840.751   
2015/01                      4156.624                 62606.691   
2015/02                      2764.168                 59385.569   
2015/03                      1981.042                 66016.590   
2015/04                      2686.685                 62053.138   
2015/05                      1764.700                 62429.804   
2015/06                      1894.086                 73052.988   
...                               ...                       ...   
2018/08                      1041.197                 47706.004   
2018/09                      1235.343                 44748.396   
2018/10                      1223.892                 56724.238   
2018/11                      1557.600                 53172.153   
2018/12                      3106.000                 50376.305   
2019/01                      2585.372                 56052.230   
2019/02                      1970.861                 57092.341   
2019/03                      1912.000                 55603.346   
2019/04                      1860.000                 56905.398   
2019/05                      1172.000                 68002.526   
2019/06                      1474.909                 61950.649   
2019/07                      1641.000                 68150.309   
2019/08                      1386.000                 67884.911   
2019/09                      1152.000                 61863.566   
2019/10                      1831.909                 72259.029   
2019/11                      1817.000                 66985.100   
2019/12                      3490.000                 73135.188   
2020/01                      2111.000                 70212.504   
2020/02                      2140.000                 66346.818   
2020/03                      1042.000                 66184.307   
2020/04                       854.485                 54118.582   
2020/05                       945.000                 54593.284   
2020/06                      1439.000                 53522.169   
2020/07                      1961.000                 74655.509   
2020/08                      1483.000                 70199.706   
2020/09                      2176.400                 73118.219   
2020/10    

In [37]:
pd.pivot_table(DF_DERIVADO_FINAL,
   index=['year_month'],
   values=['volume'],
   columns=[ 'product'],
   fill_value='',
   aggfunc=np.sum,
   margins=True,
   margins_name='Total')

volume                                           \
product    ETANOL HIDRATADO (m3) GASOLINA C (m3) GASOLINA DE AVIAÇÃO (m3)   
year_month                                                                  
2000/01             4.251875e+05    1.810572e+06                 7615.609   
2000/02             4.217064e+05    1.966365e+06                 7931.369   
2000/03             4.052662e+05    1.801198e+06                 6133.995   
2000/04             4.146097e+05    1.863562e+06                 6130.738   
2000/05             4.352146e+05    1.869176e+06                 5377.796   
2000/06             3.766284e+05    2.011256e+06                 5864.570   
2000/07             4.039528e+05    1.780834e+06                 5723.442   
2000/08             3.481598e+05    1.847763e+06                 5711.822   
2000/09             3.407314e+05    1.885878e+06                 5006.454   
2000/10             3.431272e+05    1.919653e+06                 6303.123   
2000/11             3.378746e+05    1.875821e+06                 7007.403   
2000/12             3.511294e+05    1.998117e+06                 7133.295   
2001/01             3.169596e+05    1.845440e+06                 8068.991   
2001/02             2.799114e+05    1.714464e+06                 7025.887   
2001/03             3.212576e+05    1.885719e+06                 6839.244   
2001/04             2.892016e+05    1.862995e+06                 5442.504   
2001/05             2.998034e+05    1.901299e+06                 7069.859   
2001/06             2.929488e+05    1.940547e+06                 3609.132   
2001/07             2.822584e+05    1.813032e+06                 4586.734   
2001/08             2.842366e+05    1.909391e+06                 5288.743   
2001/09             2.626666e+05    1.794803e+06                 4718.651   
2001/10             2.933950e+05    1.868048e+06                 6106.404   
2001/11             2.753239e+05    1.791888e+06                 5792.656   
2001/12             3.040306e+05    1.883374e+06                 6282.680   
2002/01             2.773112e+05    1.921459e+06                 7868.217   
2002/02             2.469572e+05    1.805728e+06                 6913.561   
2002/03             2.645817e+05    1.876689e+06                 6095.685   
2002/04             3.077612e+05    1.893523e+06                 4970.103   
2002/05             3.164889e+05    1.885042e+06                 4659.387   
2002/06             3.105217e+05    1.757909e+06                 4322.948   
...                          ...             ...                      ...   
2018/08             1.822817e+06    3.197553e+06                 3664.068   
2018/09             1.799251e+06    2.887525e+06                 4389.084   
2018/10             2.062893e+06    3.053220e+06                 3161.387   
2018/11             1.945480e+06    3.018897e+06                 2796.221   
2018/12             2.055198e+06    3.456347e+06                 5035.782   
2019/01             1.859510e+06    3.126943e+06                 4650.890   
2019/02             1.729380e+06    2.956866e+06                 3839.838   
2019/03             1.755817e+06    3.112213e+06                 3586.933   
2019/04             1.817073e+06    3.195715e+06                 3430.670   
2019/05             1.869674e+06    3.139780e+06                 2858.736   
2019/06             1.729295e+06    2.955523e+06                 3558.690   
2019/07             1.865747e+06    3.226325e+06                 4036.737   
2019/08             1.868818e+06    3.257545e+06                 3524.144   
2019/09             1.873380e+06    3.088984e+06                 3538.509   
2019/10             2.055841e+06    3.322277e+06                 3411.319   
2019/11             1.981879e+06    3.219829e+06                 3008.131   
2019/12             2.137636e+06    3.563035e+06                 3701.033   
2020/01             1.900092e+06    3.167240e+06                 3565.817   
2020/02             1.

### Modelando dados do DF_BIG_TABLE

In [38]:
DF_BIG_1 = DF_DERIVADO_FINAL.copy()
DF_BIG_2 = DF_DISEL_FINAL.copy()

In [39]:
DF_BIG_1 ['DATA'] = 'DADOS_DERIVADO' 
DF_BIG_2 ['DATA'] = 'DADOS_DISEL'

In [40]:
DF_BIG_FINAL = DF_BIG_1.append(DF_BIG_2, ignore_index=True)

### Salvando no Excel

In [41]:
dir = './new_tables'
try:
    os.makedirs(dir)
    print('diretorio criado')
except:
    print('diretorio já existe')

diretorio criado


In [42]:
# criar workbook
wb = Workbook('./new_tables/big_table.xlsx')
wb.save('./new_tables/big_table.xlsx')
wb = load_workbook('./new_tables/big_table.xlsx')


In [43]:
# adicionar workbook
sheet1 = wb.create_sheet('DF_DERIVADO_FINAL',0)
sheet2 = wb.create_sheet('DF_DISEL_FINAL',1)
sheet3 = wb.create_sheet('DF_BIG_FINAL',2)


In [44]:
# Remove existing sheet
ref = wb['Sheet']
wb.remove(ref)

In [45]:
# Seleciona sheet e exporta os dados

active = wb['DF_DERIVADO_FINAL']
for x in dataframe_to_rows(DF_DERIVADO_FINAL, index = False):
     active.append(x)
        
active = wb['DF_DISEL_FINAL']
for x in dataframe_to_rows(DF_DISEL_FINAL, index = False):
     active.append(x)

active = wb['DF_BIG_FINAL']
for x in dataframe_to_rows(DF_BIG_FINAL, index = False):
     active.append(x)


In [46]:
# Salva a excel
wb.save('./new_tables/big_table.xlsx')